In [39]:
from clickhouse_driver import Client

In [40]:
ch = Client(
    host='localhost',
    port=9000,
    user='admin',
    password='admin'
)

## Создание и загрузка таблиц, базовая аналитика

Создайте таблицу с данными из открытых источников, например, с информацией о поездках такси в Нью-Йорке с разными типами колонок: DateTime, UInt32, String.

Загрузите данные с помощью SQL-запросов INSERT или через clickhouse-client.

Выполните запросы по выборке, агрегациям, фильтрации (например, подсчёт общего количества поездок за каждый день, среднюю стоимость).
Это поможет освоить создание таблиц, типы данных, базовые операции SELECT, GROUP BY.

In [41]:
ch.execute("DROP TABLE IF EXISTS taxi_rny;")
ch.execute(
    '''
    CREATE TABLE IF NOT EXISTS taxi_rny (
        pickup_datetime DateTime,
        dropoff_datetime DateTime,
        passenger_count UInt32,
        trip_distance Float32,
        payment_type String,
        fare_amount Float32,
        tip_amount Float32,
        total_amount Float32,
        pickup_location String,
        dropoff_location String
    ) ENGINE = MergeTree()
    ORDER BY pickup_datetime;
    '''
)

[]

- Вставка данных из файла с clickhouse-client

Если данные есть в файле (например, CSV), то можно загрузить их так:
`clickhouse-client --query="INSERT INTO user FORMAT CSV" < data.csv` 
где data.csv — файл с данными, строки разделены запятыми.

- Вставка из сжатого файла

Поддерживаются разные форматы сжатия: 
`clickhouse-client --query="INSERT INTO user FORMAT CSV" < data.csv.gz`
При этом ClickHouse автоматически определит формат сжатия.

Рекомендации по эффективности загрузки

    Вставляйте данные большими пакетами (от 1 000 до 100 000 строк за раз).

    Если данные поступают малыми порциями, используйте асинхронную вставку или буферные таблицы.

    Для асинхронной вставки включите настройку async_insert, чтобы снизить нагрузку и увеличить пропускную способность.

In [42]:
'''INSERT INTO taxi_rny
SELECT
    -- Случайное время посадки в диапазоне от 2025-01-01 до 2025-01-31
    toDateTime('2025-01-01 00:00:00') + INTERVAL number SECOND AS pickup_datetime,
    
    -- Время высадки через 5-3600 секунд после посадки
    pickup_datetime + INTERVAL randInt32(5, 3600) SECOND AS dropoff_datetime,
    
    -- Количество пассажиров 1-6
    randRange(1, 6) AS passenger_count,
    
    -- Дистанция поездки в милях, 0.5 - 30.0
    randF32() * 29.5 + 0.5 AS trip_distance,
    
    -- Случайный способ оплаты из списка
    arrayElement(['card', 'cash', 'mobile', 'voucher'], randRange(1, 4)) AS payment_type,
    
    -- Стоимость поездки 5-100 долларов
    randF32() * 95 + 5 AS fare_amount,
    
    -- Чаевые 0-20% от стоимости
    fare_amount * randF32() * 0.2 AS tip_amount,
    
    -- Общая сумма поездки (стоимость + чаевые)
    fare_amount + tip_amount AS total_amount,
    
    -- Случайные локации посадки из списка (примитивный пример)
    arrayElement(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], randRange(1,5)) AS pickup_location,
    
    -- Случайные локации высадки (может совпадать с посадкой)
    arrayElement(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], randRange(1,5)) AS dropoff_location

FROM numbers(10000)
SETTINGS max_threads = 4;'''

"INSERT INTO taxi_rny\nSELECT\n    -- Случайное время посадки в диапазоне от 2025-01-01 до 2025-01-31\n    toDateTime('2025-01-01 00:00:00') + INTERVAL number SECOND AS pickup_datetime,\n\n    -- Время высадки через 5-3600 секунд после посадки\n    pickup_datetime + INTERVAL randInt32(5, 3600) SECOND AS dropoff_datetime,\n\n    -- Количество пассажиров 1-6\n    randRange(1, 6) AS passenger_count,\n\n    -- Дистанция поездки в милях, 0.5 - 30.0\n    randF32() * 29.5 + 0.5 AS trip_distance,\n\n    -- Случайный способ оплаты из списка\n    arrayElement(['card', 'cash', 'mobile', 'voucher'], randRange(1, 4)) AS payment_type,\n\n    -- Стоимость поездки 5-100 долларов\n    randF32() * 95 + 5 AS fare_amount,\n\n    -- Чаевые 0-20% от стоимости\n    fare_amount * randF32() * 0.2 AS tip_amount,\n\n    -- Общая сумма поездки (стоимость + чаевые)\n    fare_amount + tip_amount AS total_amount,\n\n    -- Случайные локации посадки из списка (примитивный пример)\n    arrayElement(['Manhattan', 'Broo

In [ ]:
generate_query = '''
INSERT INTO taxi_rny
SELECT
    start_time AS pickup_datetime,
    start_time + INTERVAL randInt32(5, 3600) SECOND AS dropoff_datetime,
    randRange(1, 6) AS passenger_count,
    randF32() * 29.5 + 0.5 AS trip_distance,
    arrayElement(['card', 'cash', 'mobile', 'voucher'], randRange(1, 4)) AS payment_type,
    fare_amount,
    fare_amount * randF32() * 0.2 AS tip_amount,
    fare_amount + fare_amount * randF32() * 0.2 AS total_amount,
    arrayElement(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], randRange(1, 5)) AS pickup_location,
    arrayElement(['Manhattan', 'Brooklyn', 'Queens', 'Bronx', 'Staten Island'], randRange(1, 5)) AS dropoff_location
FROM
(
    SELECT toDateTime('2025-01-01 00:00:00') + INTERVAL number SECOND AS start_time,
           randF32() * 95 + 5 AS fare_amount
    FROM numbers(100000)
)
SETTINGS max_threads = 4;
'''

In [48]:
# cat ../data/taxi_rny.csv | clickhouse-client --query="INSERT INTO taxi_rny FORMAT CSVWithNames"
# используем формат CSVWithNames, если в файле есть заголовок с именами колонок. 
ch.execute('''
    INSERT INTO taxi_rny
    (pickup_datetime, dropoff_datetime, passenger_count, trip_distance, payment_type, fare_amount, tip_amount, total_amount, pickup_location, dropoff_location)
VALUES
    ('2025-01-01 08:00:00', '2025-01-01 08:15:00', 2, 5.2, 'card', 15.0, 3.0, 18.0, 'Manhattan', 'Queens'),
    ('2025-01-01 09:30:00', '2025-01-01 09:45:30', 1, 3.8, 'cash', 12.5, 2.0, 14.5, 'Brooklyn', 'Manhattan'),
    ('2025-01-02 12:00:00', '2025-01-02 12:20:00', 3, 8.5, 'mobile', 22.0, 4.0, 26.0, 'Queens', 'Bronx'),
    ('2025-01-02 13:15:10', '2025-01-02 13:45:00', 4, 10.0, 'voucher', 25.0, 0.0, 25.0, 'Bronx', 'Staten Island'),
    ('2025-01-03 07:45:00', '2025-01-03 08:05:00', 1, 6.0, 'card', 18.0, 3.5, 21.5, 'Manhattan', 'Brooklyn'),
    ('2025-01-03 10:30:00', '2025-01-03 10:50:00', 2, 7.0, 'cash', 20.0, 2.0, 22.0, 'Queens', 'Manhattan'),
    ('2025-01-04 16:00:00', '2025-01-04 16:30:00', 5, 12.5, 'mobile', 30.0, 5.0, 35.0, 'Staten Island', 'Bronx'),
    ('2025-01-05 18:10:00', '2025-01-05 18:40:00', 3, 9.5, 'card', 24.0, 4.5, 28.5, 'Brooklyn', 'Queens'),
    ('2025-01-06 20:00:00', '2025-01-06 20:30:00', 2, 8.0, 'cash', 21.0, 3.0, 24.0, 'Bronx', 'Manhattan'),
    ('2025-01-07 22:15:00', '2025-01-07 22:45:00', 1, 5.0, 'voucher', 15.0, 2.0, 17.0, 'Manhattan', 'Staten Island');
''')

[]

In [50]:
ch.execute("SELECT * FROM taxi_rny LIMIT 1;")

[(datetime.datetime(2025, 1, 1, 8, 0),
  datetime.datetime(2025, 1, 1, 8, 15),
  2,
  5.199999809265137,
  'card',
  15.0,
  3.0,
  18.0,
  'Manhattan',
  'Queens')]

In [ ]:
# Подсчёт общего количества поездок за каждый день: 
query = '''
        SELECT toDate(pickup_datetime) AS day, COUNT(*) AS trip_count
        FROM taxi_rny
        GROUP BY day
        ORDER BY day;
        '''
ch.execute(query)

[(datetime.date(2025, 1, 1), 2),
 (datetime.date(2025, 1, 2), 2),
 (datetime.date(2025, 1, 3), 2),
 (datetime.date(2025, 1, 4), 1),
 (datetime.date(2025, 1, 5), 1),
 (datetime.date(2025, 1, 6), 1),
 (datetime.date(2025, 1, 7), 1)]

In [52]:
# Средняя стоимость поездки (total_amount) по дням: 
query = '''
        SELECT toDate(pickup_datetime) AS day, avg(total_amount) AS avg_fare
        FROM taxi_rny
        GROUP BY day
        ORDER BY day;
        '''
ch.execute(query)

[(datetime.date(2025, 1, 1), 16.25),
 (datetime.date(2025, 1, 2), 25.5),
 (datetime.date(2025, 1, 3), 21.75),
 (datetime.date(2025, 1, 4), 35.0),
 (datetime.date(2025, 1, 5), 28.5),
 (datetime.date(2025, 1, 6), 24.0),
 (datetime.date(2025, 1, 7), 17.0)]

In [53]:
# Фильтрация: поездки с количеством пассажиров больше 2 и стоимостьми выше 20:
query = '''
        SELECT pickup_datetime, passenger_count, total_amount
        FROM taxi_rny
        WHERE passenger_count > 2 AND total_amount > 20
        ORDER BY pickup_datetime
        LIMIT 10;
        '''
ch.execute(query)

[(datetime.datetime(2025, 1, 2, 12, 0), 3, 26.0),
 (datetime.datetime(2025, 1, 2, 13, 15, 10), 4, 25.0),
 (datetime.datetime(2025, 1, 4, 16, 0), 5, 35.0),
 (datetime.datetime(2025, 1, 5, 18, 10), 3, 28.5)]